# How to migrate from legacy LangChain agents to LangGraph

Here we focus on how to move from legacy LangChain agents to LangGraph agents.
LangChain agents (the
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor)
in particular) have multiple configuration parameters. In this notebook we will
show how those parameters map to the LangGraph
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent).

#### Prerequisites

This how-to guide uses Claude (model: "claude-3-haiku-20240307") as the LLM. Set
your Anthropic API key to run.

In [1]:
// Deno.env.set("ANTHROPIC_API_KEY", "sk-...");

// Optional, add tracing in LangSmith
// Deno.env.set("LANGCHAIN_API_KEY", "ls...");
Deno.env.set("LANGCHAIN_CALLBACKS_BACKGROUND", "true");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");
Deno.env.set("LANGCHAIN_PROJECT", "How to migrate: LangGraphJS");


## Basic Usage

For basic creation and usage of a tool-calling ReAct-style agent, the
functionality is the same. First, let's define a model and tool(s), then we'll
use those to create an agent.

In [2]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";
import { ChatAnthropic } from "@langchain/anthropic";

const llm = new ChatAnthropic({ model: "claude-3-haiku-20240307" });

const magicTool = new DynamicStructuredTool({
  name: "magic_function",
  description: "Applies a magic function to an input.",
  schema: z.object({
    input: z.number(),
  }),
  func: async ({ input }: { input: number }) => {
    return `${input + 2}`;
  },
});

const tools = [magicTool];

const query = "what is the value of magic_function(3)?";


For the LangChain
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor),
we define a prompt with a placeholder for the agent's scratchpad. The agent can
be invoked as follows:

In [3]:
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { createToolCallingAgent } from "langchain/agents";
import { AgentExecutor } from "langchain/agents";

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant"],
  ["human", `{input}`],
  new MessagesPlaceholder("agent_scratchpad"),
]);

const agent = createToolCallingAgent({ llm, tools, prompt });
const agentExecutor = new AgentExecutor({ agent, tools });

await agentExecutor.invoke({ input: query });


{
  input: "what is the value of magic_function(3)?",
  output: "The value of `magic_function(3)` is 5."
}

LangGraph's
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent)
manages a state that is defined by a list of messages. It will continue to
process the list until there are no tool calls in the agent's output. To kick it
off, we input a list of messages. The output will contain the entire state of
the graph-- in this case, the conversation history.

In [5]:
import { createReactAgent } from "@langchain/langgraph/prebuilt";

const app = createReactAgent(llm, tools);

let messages = app.invoke({ messages: [new HumanMessage(query)] });

// {  
//     input: query,
//     output: messages.messages[messages.messages.length - 1].content,
// }

TypeError: createReactAgent is not a function

In [ ]:
const messageHistory = messages.messages;
const newQuery = "Pardon?";

messages = app.invoke({ messages: [...messageHistory, new HumanMessage(newQuery)] });

{
    input: newQuery,
    output: messages.messages[messages.messages.length - 1].content,
}

## Prompt Templates

With legacy LangChain agents you have to pass in a prompt template. You can use
this to control the agent.

With LangGraph
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent),
by default there is no prompt. You can achieve similar control over the agent in
a few ways:

1. Pass in a system message as input
2. Initialize the agent with a system message
3. Initialize the agent with a function to transform messages before passing to
   the model.

Let's take a look at all of these below. We will pass in custom instructions to
get the agent to respond in Spanish.

First up, using AgentExecutor:

In [ ]:
const spanishPrompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant. Respond only in Spanish."],
  ["human", `{input}`],
  new MessagesPlaceholder("agent_scratchpad"),
]);

const spanishAgent = createToolCallingAgent(model, tools, spanishPrompt);
const spanishAgentExecutor = new AgentExecutor({ agent: spanishAgent, tools });

spanishAgentExecutor.invoke({ input: query });


Now, let's pass a custom system message to
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent).
This can either be a string or a LangChain SystemMessage.

In [ ]:
import { SystemMessage } from "@langchain/core/messages";

const systemMessage = "You are a helpful assistant. Respond only in Spanish.";

// This could also be a SystemMessage object
// const systemMessage = new SystemMessage("You are a helpful assistant. Respond only in Spanish.");

const appWithSystemMessage = createReactAgent(llm, tools, {
  messagesModifier: systemMessage,
});

messages = appWithSystemMessage.invoke({ messages: [new HumanMessage(query)] });


We can also pass in an arbitrary function. This function should take in a list
of messages and output a list of messages. We can do all types of arbitrary
formatting of messages here. In this cases, let's just add a SystemMessage to
the start of the list of messages.

In [ ]:
const modifyMessages = (messages: BaseMessage[]) => {
  const systemPrompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant. Respond only in Spanish."],
    new MessagesPlaceholder("messages"),
  ]);

  const modifiedMessages = systemPrompt.invoke({ messages }).toMessages();
  modifiedMessages.push(
    new HumanMessage("Also say 'Pandamonium!' after the answer."),
  );
  return modifiedMessages;
};

const appWithMessagesModifier = createReactAgent(llm, tools, {
  messagesModifier: modifyMessages,
});

messages = appWithMessagesModifier.invoke({
  messages: [new HumanMessage(query)],
});

console.log({
  input: query,
  output: messages.messages[messages.messages.length - 1].content,
});

## Memory

With LangChain's
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.iter),
you could add chat
[Memory](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.memory)
so it can engage in a multi-turn conversation.

In [ ]:
import { ChatMessageHistory } from "@langchain/core/memory";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";

const memory = new ChatMessageHistory({ sessionId: "test-session" });
const agentExecutorWithMemory = new RunnableWithMessageHistory({
  runnable: agentExecutor,
  memoryFactory: () => memory,
  inputMessagesKey: "input",
  historyMessagesKey: "chat_history",
});

const config = { configurable: { session_id: "test-session" } };

console.log(
  agentExecutorWithMemory.invoke(
    { input: "Hi, I'm polly! What's the output of magic_function of 3?" },
    config,
  ).output,
);
console.log("---");
console.log(
  agentExecutorWithMemory.invoke({ input: "Remember my name?" }, config).output,
);
console.log("---");
console.log(
  agentExecutorWithMemory.invoke(
    { input: "what was that output again?" },
    config,
  ).output,
);

#### In LangGraph

Memory is just
[persistence](https://langchain-ai.github.io/langgraph/how-tos/persistence/),
aka
[checkpointing](https://langchain-ai.github.io/langgraph/reference/checkpoints/).

Add a `checkpointer` to the agent and you get chat memory for free.

In [ ]:
import { MemorySaver } from "@langchain/langgraph/memory";

const memory = new MemorySaver();
const appWithMemory = createReactAgent(model, tools, {
  messagesModifier: systemMessage,
  checkpointer: memory,
});

const config = { configurable: { thread_id: "test-thread" } };

console.log(
  appWithMemory.invoke(
    {
      messages: [
        new HumanMessage(
          "Hi, I'm polly! What's the output of magic_function of 3?",
        ),
      ],
    },
    config,
  ).messages[0].content,
);
console.log("---");
console.log(
  appWithMemory.invoke(
    { messages: [new HumanMessage("Remember my name?")] },
    config,
  ).messages[0].content,
);
console.log("---");
console.log(
  appWithMemory.invoke(
    { messages: [new HumanMessage("what was that output again?")] },
    config,
  ).messages[0].content,
);

## Iterating through steps

With LangChain's
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.iter),
you could iterate over the steps using the
[stream](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.stream)
(or async `astream`) methods or the
[iter](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.iter)
method. LangGraph supports stepwise iteration using
[stream](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.stream)

In [ ]:
for await (const step of agentExecutor.stream({ input: query })) {
  console.log(step);
}


#### In LangGraph

In LangGraph, things are handled natively using
[stream](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.stream)
or the asynchronous `astream` method.

In [ ]:
for await (
  const step of app.stream(
    { messages: [new HumanMessage(query)] },
    { streamMode: "updates" },
  )
) {
  console.log(step);
}

## `return_intermediate_steps`

Setting this parameter on AgentExecutor allows users to access
intermediate_steps, which pairs agent actions (e.g., tool invocations) with
their outcomes.

In [ ]:
const agentExecutorWithIntermediateSteps = new AgentExecutor({
  agent,
  tools,
  returnIntermediateSteps: true,
});

const result = agentExecutorWithIntermediateSteps.invoke({ input: query });

console.log(result.intermediateSteps);


By default the
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent)
in LangGraph appends all messages to the central state. Therefore, it is easy to
see any intermediate steps by just looking at the full state.

In [ ]:
messages = app.invoke({ messages: [new HumanMessage(query)] });

console.log(messages);


## `max_iterations`

`AgentExecutor` implements a `max_iterations` parameter, whereas this is
controlled via `recursion_limit` in LangGraph.

Note that in AgentExecutor, an "iteration" includes a full turn of tool
invocation and execution. In LangGraph, each step contributes to the recursion
limit, so we will need to multiply by two (and add one) to get equivalent
results.

If the recursion limit is reached, LangGraph raises a specific exception type,
that we can catch and manage similarly to AgentExecutor.

In [ ]:
const badMagicTool = new DynamicStructuredTool({
  name: "magic_function",
  description: "Applies a magic function to an input.",
  schema: z.object({
    input: z.string(),
  }),
  func: async ({ input }) => {
    return "Sorry, there was an error. Please try again.";
  },
});

const badTools = [badMagicTool];

const spanishAgentExecutorWithMaxIterations = new AgentExecutor({
  agent: createToolCallingAgent(llm, badTools, spanishPrompt),
  tools: badTools,
  verbose: true,
  maxIterations: 3,
});

spanishAgentExecutorWithMaxIterations.invoke({ input: query });


In [ ]:
import { GraphRecursionError } from "@langchain/langgraph/errors";

const RECURSION_LIMIT = 2 * 3 + 1;

const appWithBadTools = createReactAgent(llm, badTools);

try {
  for await (
    const chunk of appWithBadTools.stream(
      { messages: [new HumanMessage(query)] },
      { recursionLimit: RECURSION_LIMIT },
      { streamMode: "values" },
    )
  ) {
    console.log(chunk.messages[chunk.messages.length - 1]);
  }
} catch (error) {
  if (error instanceof GraphRecursionError) {
    console.log({
      input: query,
      output: "Agent stopped due to max iterations.",
    });
  } else {
    throw error;
  }
}

## `max_execution_time`

`AgentExecutor` implements a `max_execution_time` parameter, allowing users to
abort a run that exceeds a total time limit.

In [ ]:
const slowMagic = new DynamicStructuredTool({
  name: "magic_function",
  description: "Applies a magic function to an input.",
  schema: z.object({
    input: z.string(),
  }),
  func: async ({ input }) => {
    await new Promise((resolve) => setTimeout(resolve, 2500));
    return "Sorry, there was an error. Please try again.";
  },
});

const slowTools = [slowMagic];

const slowAgentExecutor = new AgentExecutor({
  agent: createToolCallingAgent(model, slowTools),
  tools: slowTools,
  maxExecutionTime: 2000,
  verbose: true,
});

slowAgentExecutor.invoke({ input: query });


With LangGraph's react agent, you can control timeouts on two levels.

You can set a `stepTimeout` to bound each **step**:


In [ ]:
const appWithSlowTools = createReactAgent(llm, slowTools);
appWithSlowTools.stepTimeout = 2000;

try {
  for await (
    const chunk of appWithSlowTools.stream({
      messages: [new HumanMessage(query)],
    })
  ) {
    console.log(chunk);
    console.log("------");
  }
} catch (error) {
  if (error instanceof TimeoutError) {
    console.log({
      input: query,
      output: "Agent stopped due to max iterations.",
    });
  } else {
    throw error;
  }
}

The other way to set a single max timeout for an entire run is to directly use
the deno stdlib
[Promise.any()](https://doc.deno.land/deno/stable/~/Deno.Promise.any) method.

In [ ]:
try {
  await Promise.any([
    appWithSlowTools.astream({ messages: [new HumanMessage(query)] }).collect(),
    new Promise((_, reject) =>
      setTimeout(() => reject(new TimeoutError()), 3000)
    ),
  ]);
} catch (error) {
  if (error instanceof TimeoutError) {
    console.log("Task Cancelled.");
  } else {
    throw error;
  }
}


## `early_stopping_method`

With LangChain's
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.iter),
you could configure an
[early_stopping_method](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.early_stopping_method)
to either return a string saying "Agent stopped due to iteration limit or time
limit." (`"force"`) or prompt the LLM a final time to respond (`"generate"`).

In [ ]:
const earlyStoppingAgentExecutor = new AgentExecutor({
  agent: createToolCallingAgent(model, badTools, prompt),
  tools: badTools,
  earlyStoppingMethod: "force",
  maxIterations: 1,
});

const earlyStoppingResult = earlyStoppingAgentExecutor.invoke({ input: query });
console.log("Output with earlyStoppingMethod='force':");
console.log(earlyStoppingResult.output);


#### In LangGraph

In LangGraph, you can explicitly handle the response behavior outside the agent,
since the full state can be accessed.

In [ ]:
const RECURSION_LIMIT_SHORT = 2 * 1 + 1;

try {
  for await (
    const chunk of appWithBadTools.stream(
      { messages: [new HumanMessage(query)] },
      { recursionLimit: RECURSION_LIMIT_SHORT },
      { streamMode: "values" },
    )
  ) {
    console.log(chunk.messages[chunk.messages.length - 1]);
  }
} catch (error) {
  if (error instanceof GraphRecursionError) {
    console.log({
      input: query,
      output: "Agent stopped due to max iterations.",
    });
  } else {
    throw error;
  }
}

## `trim_intermediate_steps`

With LangChain's
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor),
you could trim the intermediate steps of long-running agents using
[trim_intermediate_steps](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.trim_intermediate_steps),
which is either an integer (indicating the agent should keep the last N steps)
or a custom function.

For instance, we could trim the value so the agent only sees the most recent
intermediate step.

In [ ]:
let magicStepNum = 1;

const countingMagicTool = new DynamicStructuredTool({
  name: "magic_function",
  description: "Applies a magic function to an input.",
  schema: z.object({
    input: z.number(),
  }),
  func: async ({ input }) => {
    console.log(`Call number: ${magicStepNum}`);
    magicStepNum += 1;
    return input + magicStepNum;
  },
});

const countingTools = [countingMagicTool];

const trimSteps = (steps) => {
  // Let's give the agent amnesia
  return [];
};

const agentExecutorWithTrim = new AgentExecutor({
  agent: createToolCallingAgent(llm, countingTools, prompt),
  tools: countingTools,
  trimIntermediateSteps: trimSteps,
});

const queryWithCounts =
  "Call the magic function 4 times in sequence with the value 3. You cannot call it multiple times at once.";

for await (
  const step of agentExecutorWithTrim.stream({
    input: queryWithCounts,
  })
) {
  // no-op
}

#### In LangGraph

We can use the
[`messagesModifier`](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent)
just as before when passing in [prompt templates](#prompt-templates).

In [ ]:
let magicStepNum2 = 1;

const countingMagicTool2 = new DynamicStructuredTool({
  name: "magic_function",
  description: "Applies a magic function to an input.",
  schema: z.object({
    input: z.number(),
  }),
  func: async ({ input }) => {
    console.log(`Call number: ${magicStepNum2}`);
    magicStepNum2 += 1;
    return input + magicStepNum2;
  },
});

const countingTools2 = [countingMagicTool2];

const amnesiaMessagesModifier = (messages: BaseMessage[]) => {
  // Give the agent amnesia, only keeping the original user query
  return [new SystemMessage("You are a helpful assistant"), messages[0]];
};

const amnesiaApp = createReactAgent(model, countingTools2, {
  messagesModifier: amnesiaMessagesModifier,
});

try {
  for await (
    const step of amnesiaApp.stream(
      { messages: [new HumanMessage(queryWithCounts)] },
      { streamMode: "updates" },
    )
  ) {
    // no-op
  }
} catch (error) {
  if (error instanceof GraphRecursionError) {
    console.log("Stopping agent prematurely due to triggering stop condition");
  } else {
    throw error;
  }
}